In [1]:
from geopandas import GeoDataFrame as df
import math
from math import sin, cos, sqrt, atan2, radians
from shapely.geometry import LineString,Point
import json
%matplotlib inline
R=6373.0

In [1]:
# import redis

# pool = redis.ConnectionPool(host='busvis.cloudapp.net', port=6379, 
# db=0)
# r = redis.Redis(connection_pool=pool)
# output = r.hget('stopid','MTA_803008')
# print output

import redis
pool = redis.ConnectionPool(host='busvis.cloudapp.net', port=6379, db=10)
r = redis.Redis(connection_pool=pool)
output = r.hget('speed_distance','B38')
print output


'{"speed":{"2102":1.6702832192,"2103":2.4105080209,"2104":6.1518390227,"2105":6.7672665484,"2106":2.6783146564,"2107":4.3126354665,"2108":8.385400494,"2109":3.6830026113,"2110":2.5986754466,"2111":3.4743254537,"2112":6.6625352105,"2113":7.6674195858,"2114":4.2494086287,"2115":4.0973049862,"2116":6.6963248415,"2117":5.4210598191,"2118":6.4652254453,"2119":5.1658040432,"2120":5.3247624929,"2121":2.7237998259,"2122":9.9047483473,"2123":10.5428531242,"2124":2.4724462113,"2125":7.3635524938,"2126":6.2446162554,"2127":8.0466849585,"2128":10.6607760312,"2129":4.8855022125,"2130":5.1394990602,"2131":2.4307021363,"2132":3.9096991811,"2133":6.2218086823,"2134":3.6738732296,"2135":6.0761923,"2136":5.0412703488,"2137":1.7943330474,"2138":0.8793032235,"2139":3.8866626121,"2140":4.3871982949,"2141":8.7366556995,"2142":7.769474763,"2143":4.1776629574,"2144":0.3683495641,"2145":4.3901664905,"2146":3.8716608814,"2147":4.6890324665,"2148":3.2094176007,"2149":8.3364362571,"2150":4.3859039388,"2151":8.944

In [22]:
import csv
import os
import sys

filename = "mta_shapes_meters_2014.csv"

with open(filename) as f:
    csvReader = csv.reader(f)
    headers = next(csvReader)
    d = {}
    
    for row in csvReader:
        shapeId = row[0]
        distanceEach = float(row[1])
        cur_sid, cur_segid = shapeId.strip('\n').split('-')
        
        if (cur_sid not in d) or (cur_segid not in d[cur_sid]):
            if cur_sid not in d: 
                d[cur_sid] = {}
            d[cur_sid][cur_segid] = distanceEach
            

d_accu = {}
        
for sid,value in d.items():
    for segid, dist in value.items():
        cur_accu_dist = 0.0
        
        for n in xrange(1,(int(segid)+1)):
            cur_accu_dist += d[sid][str(n)]
            
        if (sid not in d_accu) or (segid not in d_accu[sid]):
            if sid not in d_accu: 
                d_accu[sid] = {}
            d_accu[sid][segid] = cur_accu_dist
        

def WriteDictToCSV(csv_file,csv_columns,dict_data):
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.writer(csvfile, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
            writer.writerow(csv_columns)
            for SID, value in dict_data.items():
                for SEGID, DIST in value.items():
                    writer.writerow([SID,SEGID,DIST])
                    
    except IOError as (errno, strerror):
        print("I/O error({0}): {1}".format(errno, strerror)) 

    return            

csv_columns = ['shape_id', 'segment_id' , 'meters_accumulated']
csv_file = "mta_shapes_meters_2014-ACCUMULATED-1.csv"

WriteDictToCSV(csv_file,csv_columns,d_accu)

          
        
        


In [20]:
from pandas import DataFrame as df
import pandas as pd

speedACC=df.from_csv('mta_shapes_meters_2014-ACCUMULATED.csv',sep=',',index_col=None)
speedRADU=df.from_csv('mta_shapes_meters_2014-Radu.csv',sep=',',index_col=None)

speedACC.head()


,shape_id,segment_id,meters_accumulated,all_id
0,B10149,1,1.16,B10149-1
1,B10149,2,295.08,B10149-2
2,B10149,3,433.69,B10149-3
3,B10149,4,567.17,B10149-4
4,B10149,5,853.19,B10149-5


In [21]:
#speed.head()
speed_lACC = set(speedACC['all_id'].tolist())
speed_lRADU = set(speedRADU['all_id'].tolist())

print len(speed_lACC)
print len(speed_lRADU)

36421
36421
